In [1]:
import pandas as pd
import numpy as np

C:\Users\yanhui\AppData\Local\Temp\ipykernel_15068\2162656668.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# pip install gurobipy
import gurobipy as gp
from gurobipy import GRB

options = {
    "WLSACCESSID": "31598bc4-0b23-474f-86d4-af032d437137",
    "WLSSECRET": "bd7ad348-f8d6-452e-b615-ebc88d920e41",
    "LICENSEID": 2498404,
}

In [29]:
df = pd.read_csv('data_20180119_20180420.csv',index_col=0)
df = df.iloc[:80,:]

In [30]:

# df_0102 = df[df['t0'] == '2018-01-02']
C1 = df['C1']
C2 = df['C2']

### Grid for stock price
current setting S_{T2}, S_{T1} from range [1100,1300] and 20\*20 discretization

In [31]:
def gen_Grid(S_0,epsilon,n):
    S_min = S_0-epsilon
    S_max = S_0+epsilon
    S = np.linspace(S_min, S_max, num=n+1)
    return S

In [32]:
N_j = 20
epsi = 200
S0 = df['S0'].unique()
S = [gen_Grid(s,epsi,N_j) for s in S0]

### Optimization Variables
variables to optimize: lambda_1, lambda_2, d, \Delta_1, \Delta_0

In [33]:
m = gp.Model("Option Pricing")
lamb1 = m.addVars(df.index,name="lamb1")
lamb2 = m.addVars(df.index,name="lamb2")
d = m.addVars(['d'],name="d")
delta0 = m.addVars(['delta0'],name="delta0")
delta1 = m.addVars(np.array(S).flatten(),name="delta1")

### Model Construction
objective: min$d+\sum_{i=1}^{N_1}\lambda_{i,1}\Pi_{i,1}+\sum_{i=1}^{N_2}\lambda_{i,2}\Pi_{i,2}$

constrains: for each grid point (S_i,S_j), we have

$d+\sum_{n=1}^{N_1}\lambda_{n,1}(S_i-K_{n,1})^++\sum_{n=1}^{N_2}\lambda_{n,2}(S_j-K_{n,2})^++\Delta_{0}\cdot(S_i-S_0)+\Delta_{1}^i\cdot(S_j-S_i) >= \Phi(S_i,S_j)$

Current payoff $\Phi(S_i,S_j) = S_j - S_i$

In [34]:
m.setObjective(d['d']+sum(lamb1[r]*C1[r]+lamb2[r]*C2[r] for r in df.index))
m.ModelSense = GRB.MINIMIZE

In [35]:
def payoff(S1,S2):
    return S2-S1
for i in range(len(S0)):
    S_t0 = S[i]
    df_t0 = df[df['S0']==S0[i]]
    for S_i in S_t0:
        for S_j in S_t0:
            m.addConstr(d['d']+sum(lamb1[r]*max(S_i-r,0)+lamb2[r]*max(S_j-r,0) for r in df_t0.index)+delta0['delta0']*(S_i-S0[i])+delta1[S_i]*(S_j-S_i) >= payoff(S_i,S_j),'C_{}{}'.format(S_i,S_j))
            m.update()

Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


In [36]:
m.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22000.2))

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 1764 rows, 246 columns and 43932 nonzeros
Model fingerprint: 0xd04b1bcb
Coefficient statistics:
  Matrix range     [2e-01, 4e+02]
  Objective range  [4e-01, 4e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 4e+02]
Presolve removed 229 rows and 9 columns
Presolve time: 0.02s
Presolved: 1535 rows, 237 columns, 38005 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   8.172500e+03   0.000000e+00      0s
      77    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 77 iterations and 0.04 seconds (0.03 work units)
Optimal objective  0.000000000e+00


In [37]:
delta0

{'delta0': <gurobi.Var delta0[delta0] (value 0.0)>}